<a href="https://colab.research.google.com/github/MarielaNina/-Guide-to-Advanced-LLM-Techniques-Public/blob/main/M%C3%B3dulo_3_Ensembles_de_LLMs_A_Sabedoria_das_Multid%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Módulo 3: Ensembles de LLMs - A Sabedoria das Multidões

# Introdução

Nos módulos anteriores, focamos em como aprimorar a interação com um único modelo através da engenharia de prompt. Agora, vamos explorar uma técnica poderosa emprestada do aprendizado de máquina tradicional: os ensembles.

A ideia central de um ensemble é que "várias cabeças pensam melhor que uma". Em vez de confiar na resposta de um único modelo, combinamos as predições de múltiplos "especialistas" para chegar a uma decisão final mais robusta e precisa. LLMs, apesar de seu poder, podem ser suscetíveis a vieses, erros factuais ou "alucinações". Usar um ensemble pode mitigar esses riscos e aumentar a confiabilidade geral do sistema.

Neste módulo, vamos explorar duas estratégias de ensemble, conforme detalhado no artigo:
1. Votação Majoritária: A forma mais simples de ensemble, onde consultamos vários especialistas e adotamos a resposta da maioria.
2. Negociação: Uma abordagem mais avançada que simula um debate estruturado entre modelos para refinar ideias e chegar a um consenso.

# 1. Votação Majoritária

A votação é a forma mais intuitiva de ensemble. A ideia é consultar vários "especialistas" independentes e adotar a resposta da maioria. No contexto de LLMs, esses especialistas podem ser:
1. Diferentes Modelos: Você pode fazer a mesma pergunta para o Llama 3, Mixtral e Sabiá, e depois contar os votos de cada um.
2. Diferentes Prompts: Usar vários prompts (ex: um Zero-Shot, um Few-Shot) com o mesmo modelo.
3. Múltiplas Execuções do Mesmo Modelo: Esta é a abordagem conhecida como Self-Consistency (Autoconsistência), proposta por Wang et al. (2022) [1]. Executamos o mesmo prompt várias vezes com uma temperatura > 0 para gerar respostas diversas e escolhemos a mais frequente.

Vamos implementar a abordagem de Self-Consistency, pois é a mais prática quando se tem acesso a uma única API.

![](https://github.com/Assaoka/Guide-to-Advanced-LLM-Techniques/blob/main/Imagens/sc.png?raw=1)

In [1]:
!pip install -q langchain langchain-core langchain-community langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from langchain_groq import ChatGroq
from google.colab import userdata

llm_groq = ChatGroq(
    model="gemma2-9b-it",
    api_key=userdata.get('GROQ_API_KEY'),
    temperature=0.7
)

In [3]:
from typing import Literal
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class ClassificacaoNoticia(BaseModel):
    emocao: Literal["Felicidade", "Tristeza", "Nojo", "Raiva",
                    "Medo", "Surpreza", "Desprezo"] = Field(
                    description="A emoção primária transmitida pelo texto. Estamos usando como padrão as emoções universais de ekman")

parser = PydanticOutputParser(pydantic_object=ClassificacaoNoticia)
formato = parser.get_format_instructions()


In [4]:
from langchain_core.prompts import PromptTemplate

template = """Classifique a notícia abaixo quanto a emoção.
{format_instructions}
Notícia: {noticia}
"""

prompt = PromptTemplate.from_template(
    template=template,
    partial_variables={"format_instructions": formato}
)

In [7]:
chain = prompt | llm_groq | parser
chain.invoke("Nesse cenário de instabilidade, os investidores não sabem o que fazer.")

ClassificacaoNoticia(emocao='Medo')

In [8]:
def votacao(chain, texto: str, n: int) -> ClassificacaoNoticia:
    votos = [chain.invoke(texto) for _ in range(n)]
    vals = [voto.emocao for voto in votos]
    conts = {}
    for val in vals:
        print(val)
        if val in conts:
            conts[val] += 1
        else:
            conts[val] = 1

    return ClassificacaoNoticia(emocao=max(conts, key=conts.get))


votacao(chain, "Os resultados fiscais do primeiro semestre de 2024 indicam que, no curto prazo, a meta fiscal de -0,25% do PIB é mais viável, apesar das incertezas de médio e longo prazo devido aos desafios fiscais e ao aumento das despesas obrigatórias. O desempenho das receitas líquidas foi positivo, impulsionado pelas medidas legislativas de arrecadação, mas o crescimento das despesas, especialmente com benefícios previdenciários, elevou o déficit primário para R$ 68,7 bilhões, agravando o cenário fiscal. Apesar das medidas de contenção adotadas, a sustentabilidade das contas públicas a longo prazo permanece ameaçada, exigindo soluções concretas e uma postura cautelosa para os próximos anos.", 3)

Tristeza
Tristeza
Tristeza


ClassificacaoNoticia(emocao='Tristeza')

# 2. Negociação

E se, em vez de votar de forma independente, os "especialistas" pudessem debater e refinar suas ideias? Essa é a premissa da Negociação, uma técnica que simula um processo de argumentação para resolver discordâncias. O trabalho de Sun et al. (2023) [2] explora essa ideia, mostrando que um debate estruturado pode ajudar a resolver ambiguidades.

Vamos implementar um framework de debate iterativo com dois papéis:
1. Gerador: Gera a análise e classificação inicial.
2. Discriminador: Revisa a análise do propositor, busca falhas, pontos de vista alternativos ou ambiguidades e oferece uma contra-análise.

O processo é um loop:
1. O Gerador faz uma análise inicial.
2. O Discriminador avalia a análise. Podendo concordar ou apresentar uma contra proposta.
3. O Propositor revisa sua análise com base na crítica. E pode concordar ou enviar uma contra proposta.
4. O ciclo se repete até um número máximo de rodadas ou até que um consenso seja alcançado.

In [9]:
gerador = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=userdata.get('GROQ_API_KEY'),
    temperature=0.1
)

discriminador = ChatGroq(
    model="gemma2-9b-it",
    api_key=userdata.get('GROQ_API_KEY'),
    temperature=0.1
)

In [10]:
class Negociacao(BaseModel):
    raciocinio: str = Field(description="Linha de raciocínio passo a passo do modelo até chegar em uma decisão.")
    concordo: bool = Field(description="Representa se o modelo concorda ou não com o modelo anterior.")
    emocao: Literal["Felicidade", "Tristeza", "Nojo", "Raiva",
                    "Medo", "Surpreza", "Desprezo", "Neutro"] = Field(
                    description="A emoção primária transmitida pelo texto. Estamos usando como padrão as emoções universais de ekman")

parser = PydanticOutputParser(pydantic_object=Negociacao)
formato = parser.get_format_instructions()

In [11]:
gerador1_template = """Classifique a notícia abaixo quanto a emoção.
{format_instructions}
Notícia: {noticia}
"""

gerador1_prompt = PromptTemplate.from_template(
    template=gerador1_template,
    partial_variables={"format_instructions": formato}
)

In [12]:
discriminador1_template = """Você está em um debate com outro agente. A tarefa de vocês é classificar a notícia abaixo quanto a emoção.
{format_instructions}
Notícia: {noticia}
Gerador: {gerador}
"""

discriminador1_prompt = PromptTemplate.from_template(
    template=discriminador1_template,
    partial_variables={"format_instructions": formato}
)

In [13]:
iterações_template = """Você está em um debate com outro agente. A tarefa de vocês é classificar a notícia abaixo quanto a emoção.
{format_instructions}
Notícia: {noticia}
sua proposta: {analise}
Contra proposta: {contra_proposta}
"""

iterações_prompt = PromptTemplate.from_template(
    template=iterações_template,
    partial_variables={"format_instructions": formato}
)

In [14]:
inicio = gerador1_prompt | gerador | parser
discriminar = discriminador1_prompt | discriminador | parser
iterações1 = iterações_prompt | gerador | parser
iterações2 = iterações_prompt | discriminador | parser

In [15]:
def negociacao(noticia: str, n: int = 3):
    historico = [inicio.invoke({"noticia": noticia})]
    historico.append(discriminar.invoke({"noticia": noticia, "gerador": historico[-1].raciocinio}))
    for i in range(n-2):
        if historico[-1].concordo:
            break
        if i % 2 == 0:
            historico.append(iterações1.invoke({"noticia": noticia,
                                                "analise": historico[-2].raciocinio,
                                                "contra_proposta": historico[-1].raciocinio}))
        else:
            historico.append(iterações2.invoke({"noticia": noticia,
                                                "analise": historico[-2].raciocinio,
                                                "contra_proposta": historico[-1].raciocinio}))
    return historico


In [16]:
historico = negociacao("Os resultados fiscais do primeiro semestre de 2024 indicam que, no curto prazo, a meta fiscal de -0,25% do PIB é mais viável, apesar das incertezas de médio e longo prazo devido aos desafios")

In [17]:
for i, message in enumerate(historico):
    print(f'{i + 1}° Iteração:')
    role = 'Gerador' if i % 2 == 0 else 'Discriminador'
    print(f'- {role}: {message.raciocinio}')
    if i > 0:
        print(f'- Concordo: {message.concordo}')
    print(f'- Emoção: {message.emocao}')
    print()


1° Iteração:
- Gerador: Os resultados fiscais do primeiro semestre de 2024 indicam que, no curto prazo, a meta fiscal de -0,25% do PIB é mais viável, apesar das incertezas de médio e longo prazo devido aos desafios
- Emoção: Neutro

2° Iteração:
- Discriminador: A notícia apresenta dados positivos sobre os resultados fiscais do primeiro semestre de 2024, indicando que a meta fiscal de -0,25% do PIB é mais viável no curto prazo. No entanto, também menciona incertezas de médio e longo prazo devido a desafios, o que sugere uma perspectiva mais cautelosa.
- Concordo: True
- Emoção: Neutro



# 3. Atividade Prática: Construindo um Ensemble Híbrido

Nas seções anteriores, implementamos a Votação Majoritária usando Self-Consistency (múltiplas chamadas ao mesmo modelo com o mesmo prompt). Agora, vamos elevar o nível e construir um sistema mais robusto: um Ensemble Híbrido.

A ideia é aumentar a diversidade dos "eleitores". Em vez de perguntar a mesma coisa para a mesma pessoa várias vezes, vamos consultar diferentes especialistas (modelos), fazendo a eles diferentes tipos de perguntas (prompts) com a mesma questão.


![](https://github.com/Assaoka/Guide-to-Advanced-LLM-Techniques/blob/main/Imagens/HybridEnsemble.png?raw=1)

Seu objetivo: Implementar uma função de votação para um ensemble híbrido que utiliza múltiplos modelos e múltiplos prompts para classificar uma notícia.

Passos:
1. Crie Prompts Variados: Crie dois PromptTemplate distintos para a tarefa de classificação de emoção.
    - Prompt A (Direto): Pode ser similar ao que já usamos, pedindo a classificação de forma direta.
    - Prompt B (Persona): Crie um prompt que instrua o LLM a agir com uma persona específica. Ex: "Aja como um analista de risco cético e experiente. Analise a emoção primária que um investidor sentiria ao ler a seguinte notícia...". Isso força o modelo a avaliar o texto de uma perspectiva diferente.
2. Teste e Compare: Execute sua nova função com a notícia ambígua do "Dilema do Investidor". Compare o resultado e a diversidade dos votos com a abordagem de Self-Consistency. O resultado foi mais nuançado?

# Referências

[1] F. Trad and A. Chehab, To Ensemble or Not: Assessing Majority Voting Strategies for Phishing Detection with Large Language Models. Springer Nature Switzerland, 2025, p. 158–173. [Online]. Available: http://dx.doi.org/10.1007/978-3-031-82150-9 13

[2] X. Wang, J. Wei, D. Schuurmans, Q. Le, E. Chi, S. Narang, A. Chowdhery, and D. Zhou, “Self-consistency improves chain of thought reasoning in language models,” 2023. [Online]. Available: https://arxiv.org/abs/2203.11171

[3] X. Sun, X. Li, S. Zhang, S. Wang, F. Wu, J. Li, T. Zhang,  and G. Wang, “Sentiment analysis through llm negotiations,” 2023. [Online]. Available: https://arxiv.org/abs/2311.01876
